In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/colab_projects/recommendation_system/nbs/

/content/gdrive/MyDrive/colab_projects/recommendation_system/nbs


In [ ]:
import sys
sys.path.append("../src/")
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from config import *

In [ ]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [ ]:
df_train = pd.read_parquet("../data/processed/df_rating_movie_train.parquet")
df_val = pd.read_parquet("../data/processed/df_rating_movie_val.parquet")

In [ ]:
movies_ids = list(set(list(df_train.movieId.unique()) + list(df_val.movieId.unique())))

In [ ]:
users_ids = list(set(list(df_train.userId.unique()) + list(df_val.userId.unique())))

In [ ]:
len(movies_ids)

59047

In [ ]:
max(movies_ids)

209171

In [ ]:
dict_movies = {}
index = 0
for ids in sorted(movies_ids):
    dict_movies[ids] = index
    index += 1

In [ ]:
dict_users = {}
index = 0
for ids in sorted(users_ids):
    dict_users[ids] = index
    index += 1

In [ ]:
df_train["movieId"] = df_train["movieId"].map(dict_movies)
df_val["movieId"] = df_val["movieId"].map(dict_movies)

df_train["userId"] = df_train["userId"].map(dict_users)
df_val["userId"] = df_val["userId"].map(dict_users)

In [ ]:
df_train.describe()

,userId,movieId,rating,timestamp
count,2.000008e+07,2.000008e+07,2.000008e+07,2.000008e+07
mean,8.118888e+04,6.107920e+03,3.533854e+00,1.215613e+09
std,4.679203e+04,8.262760e+03,1.060744e+00,2.268744e+08
min,0.000000e+00,0.000000e+00,5.000000e-01,7.896520e+08
25%,4.051000e+04,1.167000e+03,3.000000e+00,1.011760e+09
50%,8.091300e+04,2.855000e+03,3.500000e+00,1.198869e+09
75%,1.215560e+05,7.920000e+03,4.000000e+00,1.447207e+09
max,1.625400e+05,5.904600e+04,5.000000e+00,1.574328e+09


In [ ]:
df_train.head()

,userId,movieId,rating,timestamp,title,genres
5355991,34759,4857,4.0,1008626350,Ocean's Eleven (2001),Crime|Thriller
16979199,110105,1154,1.5,1301889569,"English Patient, The (1996)",Drama|Romance|War
14781790,95710,4466,2.0,1009726295,Bill & Ted's Excellent Adventure (1989),Adventure|Comedy|Sci-Fi
5159343,33522,1373,4.0,1039513864,Hamlet (1996),Crime|Drama|Romance
18503349,119882,9979,4.0,1353051590,Cinderella Man (2005),Drama|Romance


In [ ]:
df_train.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
title         object
genres        object
dtype: object

In [ ]:
for col in ["userId", "movieId", "rating"]:
    df_train[col] = df_train[col].astype(np.float32)
    df_val[col] = df_val[col].astype(np.float32)

In [ ]:
num_unique_users=len(set(list(df_train.userId.unique()) + list(df_val.userId.unique())))
num_unique_movies=len(set(list(df_train.movieId.unique()) + list(df_val.movieId.unique())))

In [ ]:
min_rating=min(df_train.rating.min(), df_val.rating.min())
max_rating=max(df_train.rating.max(), df_val.rating.max())

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import optimizers as opt
from tensorflow.keras.layers import Embedding, multiply, concatenate, Flatten, Input, Dense

In [ ]:
users_input = Input(shape=(1,), name="users_input")
users_embedding = Embedding(num_unique_users + 1, EMBEDDING_SIZE, name="users_embeddings")(users_input)
users_bias = Embedding(num_unique_users + 1, 1, name="users_bias")(users_input)

movies_input = Input(shape=(1,), name="movies_input")
movies_embedding = Embedding(num_unique_movies + 1, EMBEDDING_SIZE, name="movies_embedding")(movies_input)
movies_bias = Embedding(num_unique_movies + 1, 1, name="movies_bias")(movies_input)

dot_product_users_movies = multiply([users_embedding, movies_embedding])

input_terms = dot_product_users_movies + users_bias + movies_bias

input_terms = Flatten(name="fl_inputs")(input_terms)

output = Dense(1, activation="relu", name="output")(input_terms)

In [ ]:
model = Model(inputs=[users_input, movies_input], outputs=output)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
users_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
movies_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
users_embeddings (Embedding)    (None, 1, 50)        8127100     users_input[0][0]                
__________________________________________________________________________________________________
movies_embedding (Embedding)    (None, 1, 50)        2952400     movies_input[0][0]               
______________________________________________________________________________________________

In [ ]:
opt_adam = opt.Adam(lr = 0.005)
model.compile(optimizer=opt_adam, loss= ['mse'], metrics=['mean_absolute_error'])

In [ ]:
model.fit(x=[df_train.userId, df_train.movieId], y=df_train.rating, batch_size=512, epochs=3, verbose=1,    
          validation_data=([df_val.userId, df_val.movieId], df_val.rating))

Epoch 1/3
39063/39063 [==============================] - 2303s 59ms/step - loss: 0.9431 - mean_absolute_error: 0.7221 - val_loss: 0.6360 - val_mean_absolute_error: 0.6075
Epoch 2/3
39063/39063 [==============================] - 2298s 59ms/step - loss: 0.5564 - mean_absolute_error: 0.5623 - val_loss: 0.6213 - val_mean_absolute_error: 0.5936
Epoch 3/3
39063/39063 [==============================] - 2298s 59ms/step - loss: 0.4777 - mean_absolute_error: 0.5171 - val_loss: 0.6310 - val_mean_absolute_error: 0.5989


In [ ]:
model.predict([df_val.userId, df_val.movieId])

array([[3.6233797],
       [4.034534 ],
       [4.375308 ],
       ...,
       [3.176009 ],
       [4.443116 ],
       [3.9021013]], dtype=float32)

In [ ]:
pd.DataFrame(model.predict([df_val.userId, df_val.movieId])).describe()

,0
count,5.000019e+06
mean,3.570453e+00
std,8.503246e-01
min,0.000000e+00
25%,3.071305e+00
50%,3.652218e+00
75%,4.161022e+00
max,7.183237e+00


In [ ]:
model.evaluate([df_val.userId, df_val.movieId], df_val.rating)

156251/156251 [==============================] - 246s 2ms/step - loss: 0.6310 - mean_absolute_error: 0.5989


[0.6310396790504456, 0.598869264125824]

In [ ]:
model.save_weights("../model/neural_collab_filtering_v1.h5")

In [ ]:
df_val["predict"] = model.predict([df_val.userId, df_val.movieId])

In [ ]:
df_val["predict"].describe()

count    5.000019e+06
mean     3.570453e+00
std      8.503246e-01
min      0.000000e+00
25%      3.071305e+00
50%      3.652218e+00
75%      4.161022e+00
max      7.183237e+00
Name: predict, dtype: float64

In [ ]:
from sklearn.metrics import mean_squared_error as mse
mse(df_val["rating"], df_val["predict"].map(lambda x: min_rating if x <= min_rating else max_rating if x >= max_rating else x))

0.6259002100957272